## 과제 4: 신경망을 사용하여 다층 퍼셉트론 모델 생성 및 실행

이번 과제에서는 강력한 ML 라이브러리에서 나온 추상화된 방법을 사용하지 않고 신경망을 생성합니다. 하지만, MXNet의 몇몇 기본 함수를 사용하면 이러한 신경망을 쉽게 모델링할 수 있습니다.

가장 단순한 신경망으로 쉽게 표현할 수 있는 **로지스틱 회귀 분석** 문제에 초점을 맞춥니다. 결과\(레이블\)가 알려진 모조 데이터 세트를 생성합니다. 필기 숫자의 28x28 흑백 사진을 반으로 자른 사진으로 구성된 [MNIST 데이터 세트](http://yann.lecun.com/exdb/mnist/)를 사용합니다.

![](https://us-west-2-tcprod.s3.amazonaws.com/courses/ILT-TF-200-MLDEEP/v1.1.2/lab-1-setup-ami-DL/instructions/en_us/images/mnistdigits.gif)

숫자를 감지하기 위해 4계층 신경망을 생성합니다.

다음 코드를 Jupyter 노트북의 **In** 셀에 붙여넣어 노트북 셀로 종속성을 가져옵니다.

In [1]:
# Import dependencies
from __future__ import print_function
import mxnet as mx
import numpy as np
from mxnet import nd, autograd
print("Dependencies imported")

Dependencies imported


**Shift+Enter** 키를 눌러 셀을 실행합니다. 셀 실행이 완료되면 셀 좌측의 숫자가 **In \[\*\]:** 에서 **In \[1\]** 로 변경됩니다.

MXNet에서 GPU를 사용하여 지정하려면, 다음번 셀에서 다음 명령을 실행합니다.

In [2]:
# Use a GPU with MXNet
ctx = mx.gpu()

다층 퍼셉트론 신경망을 위해 [MNIST 이미지 데이터 세트](http://yann.lecun.com/exdb/mnist/)를 사용합니다. 데이터 세트를 다운로드하려면, 다음번 셀에서 다음을 입력합니다.

In [3]:
# Get the MNIST image dataset
mnist = mx.test_utils.get_mnist()

신경망에 대한 파라미터를 정의하려면 다음번 셀에 다음 코드를 붙여넣습니다.

In [4]:
# Parameters for the neural network

# Number of inputs: A 1-dimensional input consisting of a single image (28 pixels by 28 pixels)
num_inputs = 784

# Number of Outputs: Number of outputs to be predicted by the network (Digits 0-9)
num_outputs = 10

# Batch size is the number of images processed in a single batch
batch_size = 64

파라미터를 정의한 후에 데이터 세트를 교육 데이터와 테스트 데이터로 나눌 수 있습니다. 데이터 세트를 나누려면 새 셀에서 다음을 실행합니다.

In [5]:
# Split the dataset into training data dn test data

def transform(data, label):
    return data.astype(np.float32)/255, label.astype(np.float32)

train_data = mx.gluon.data.DataLoader(mx.gluon.data.vision.MNIST(train=True, transform=transform),batch_size, shuffle=True)
test_data = mx.gluon.data.DataLoader(mx.gluon.data.vision.MNIST(train=False, transform=transform),batch_size, shuffle=False)

숨겨진 뉴런 수 및 가중치 배율 등의 유용한 파라미터를 정의하려면, 다음번 셀에서 다음을 실행합니다.

In [6]:
# Number of hidden neurons
num_hidden = 256

# Weights scale
weight_scale = .01

**첫 번째 계층**에 대한 파라미터\(가중치 및 편중치\)를 할당하려면, 다음번 셀에서 다음 코드를 실행합니다.

In [7]:
# Allocate weights and bias for the first layer
w_hd_1 = nd.random_normal(shape=(num_inputs, num_hidden), scale=weight_scale, ctx=ctx)
b_hd_1 = nd.random_normal(shape=num_hidden, scale=weight_scale, ctx=ctx)

**두 번째 계층**에 대한 파라미터\(가중치 및 편중치\)를 할당하려면, 다음번 셀에서 다음 코드를 실행합니다.

In [8]:
# Allocate weights and bias for the second layer
w_hd_2 = nd.random_normal(shape=(num_hidden, num_hidden), scale=weight_scale, ctx=ctx)
b_hd_2 = nd.random_normal(shape=num_hidden, scale=weight_scale, ctx=ctx)

**출력 계층**에 대한 파라미터\(가중치 및 편중치\)를 할당하려면, 다음번 셀에서 다음 코드를 실행합니다.

In [9]:
# Allocate weights and bias for the output layer
w_output = nd.random_normal(shape=(num_hidden, num_outputs), scale=weight_scale, ctx=ctx)
b_output = nd.random_normal(shape=num_outputs, scale=weight_scale, ctx=ctx)

기울기를 계산할 수 있도록 목록에 파라미터를 추가하려면, 다음번 셀에서 다음 코드를 실행합니다.

In [10]:
# Add parameters to calculate gradients
params = [w_hd_1, b_hd_1, w_hd_2, b_hd_2, w_output, b_output]


for param in params:
    param.attach_grad()

선형 뉴런으로 다중 계층 네트워크를 정의하면, 해당 네트워크는 선형 함수만 될 수 있습니다. 네트워크에서 비선형 속성을 포착하도록 하려면, 계층 끝에 활성화 함수를 추가해야 합니다. 이 경우 ReLU 활성화 함수를 사용할 수 있습니다.

은닉층에 대한 ReLU 활성화 함수를 정의하려면, 다음번 셀에서 다음 코드를 실행합니다.


In [11]:
# Define a ReLU activiation function for the hidden layer
def relu(X):
    return nd.maximum(X, nd.zeros_like(X))

Sigmoid나 Tanh 등의 다른 활성화 함수를 사용할 수도 있습니다.

네트워크의 출력에서 예측은 숫자 0\~9를 예측하는 배열이 됩니다. 출력 계층을 위한 Softmax 활동 함수는 이미지가 특정 클래스에 속하는 확률을 제공합니다. 예를 들어, 배열의 첫 숫자가 0.65라면, 숫자가 0일 확률이 65%라는 뜻입니다.

In [12]:
# Use a softmax action function for the output layer
def softmax_cross_entropy(yhat_linear, y):
    return - nd.nansum(y * nd.log_softmax(yhat_linear), axis=0, exclude=True)

---
## 과제 5: 인공 신경망 모델 정의

이번 과제에서는 인공 신경망, 가중치와 편중치를 학습하기 위한 최적화 도구 및 모델 수행 방식을 평가하는 평가 지표를 정의합니다.

모델을 정의하려면, 새 셀에서 다음을 실행합니다.

In [13]:
# Neural network model
def net(X):

    #  Compute the first hidden layer
    h1_linear = nd.dot(X, w_hd_1) + b_hd_1
    h1 = relu(h1_linear)

    #  Compute the second hidden layer
    h2_linear = nd.dot(h1, w_hd_2) + b_hd_2
    h2 = relu(h2_linear)

    #  Compute the output layer.
    yhat_linear = nd.dot(h2, w_output) + b_output
    return yhat_linear

반환된 변수는 여전히 Softmax 함수가 적용되지 않은 선형 변수입니다. 해당 함수는 역 전달 중 발생할 수 있는 수치 안정성 문제를 방지하기 위해 softamx\_cross\_entropy에 직접 적용됩니다.

가중치와 편중치를 학습하기 위한 최적화 도구를 정의하려면 새 셀에서 다음을 실행합니다.


In [14]:
# Optimizer
def SGD(params, lr):
    for param in params:
        param[:] = param - lr * param.grad

평가 지표를 정의하려면, 새 셀에서 다음을 실행합니다.

In [15]:
# Evaluation metric
def evaluate_accuracy(data_iterator, net):
    numerator = 0.
    denominator = 0.
    for i, (data, label) in enumerate(data_iterator):
        data = data.as_in_context(ctx).reshape((-1, 784))
        label = label.as_in_context(ctx)
        output = net(data)
        predictions = nd.argmax(output, axis=1)
        numerator += nd.sum(predictions == label)
        denominator += data.shape[0]
    return (numerator / denominator).asscalar()

___
## 과제 6: 교육 루프 및 모델 평가 실행
교육 루프 실행을 위한 파라미터를 정의합니다.

In [16]:
# Epochs are iterations over the full network
epochs = 10

# Learning rate parameter determines the speed at which the network learns
learning_rate = 0.001

# Defining a smooth constant for the moving loss
smoothing_constant = 0.01

인공 신경망 모델을 학습합니다.

In [17]:
# Train the neural network model
for e in range(epochs):
    for i, (data, label) in enumerate(train_data):
        data = data.as_in_context(ctx).reshape((-1, 784))
        label = label.as_in_context(ctx)
        label_one_hot = nd.one_hot(label, 10)
        with autograd.record():
            output = net(data)
            loss = softmax_cross_entropy(output, label_one_hot)
        loss.backward()
        SGD(params, learning_rate)

        ##########################
        #  Keep a moving average of the losses
        ##########################
        curr_loss = nd.mean(loss).asscalar()
        moving_loss = (curr_loss if ((i == 0) and (e == 0))
                       else (1 - smoothing_constant) * moving_loss + (smoothing_constant) * curr_loss)

    test_accuracy = evaluate_accuracy(test_data, net)
    train_accuracy = evaluate_accuracy(train_data, net)
    print("Epoch %s. Loss: %s, Train_acc %s, Test_acc %s" %
          (e, moving_loss, train_accuracy, test_accuracy))

Epoch 0. Loss: 0.4629979529702352, Train_acc 0.88315, Test_acc 0.8838
Epoch 1. Loss: 0.28015616851090236, Train_acc 0.91735, Test_acc 0.9192
Epoch 2. Loss: 0.19353834659481542, Train_acc 0.9482667, Test_acc 0.9463
Epoch 3. Loss: 0.14636655652063246, Train_acc 0.9604667, Test_acc 0.9576
Epoch 4. Loss: 0.1212956941057802, Train_acc 0.96936667, Test_acc 0.9649
Epoch 5. Loss: 0.09707598250554723, Train_acc 0.9759333, Test_acc 0.9691
Epoch 6. Loss: 0.08412505601372673, Train_acc 0.9802833, Test_acc 0.9726
Epoch 7. Loss: 0.07681503703772326, Train_acc 0.98118335, Test_acc 0.9725
Epoch 8. Loss: 0.06473273171303814, Train_acc 0.98441666, Test_acc 0.9749
Epoch 9. Loss: 0.06033490354394843, Train_acc 0.98695, Test_acc 0.9751


보이지 않는 데이터에 대한 모델의 작동 방식을 확인하려면 새 셀에서 다음을 실행하여 Jupyter 노트북에 그릴 수 있는 캔버스가 포함된 html 페이지를 다운로드합니다.

In [18]:
%%bash
wget http://us-west-2-tcprod.s3.amazonaws.com/courses/ILT-TF-200-MLDEEP/v1.1.1/lab-1-setup-ami-DL/scripts/mnist.html

--2018-11-18 13:38:04--  http://us-west-2-tcprod.s3.amazonaws.com/courses/ILT-TF-200-MLDEEP/v1.1.1/lab-1-setup-ami-DL/scripts/mnist.html
Resolving us-west-2-tcprod.s3.amazonaws.com (us-west-2-tcprod.s3.amazonaws.com)... 52.218.216.170
Connecting to us-west-2-tcprod.s3.amazonaws.com (us-west-2-tcprod.s3.amazonaws.com)|52.218.216.170|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2970 (2.9K) [text/html]
Saving to: ‘mnist.html’

     0K ..                                                    100%  379M=0s

2018-11-18 13:38:05 (379 MB/s) - ‘mnist.html’ saved [2970/2970]



Jupyter 노트북에는 매직 명령어\(Magic Command\)가 내장되어 있어 Jupyter 노트북 셀에서 Bash 명령어 및 기타 명령어를 실행할 수 있습니다. 자세한 내용은 [Magic Commands](http://ipython.readthedocs.io/en/stable/interactive/magics.html)를 참조하십시오.

다음 셀을 실행하여 모델을 평가하기 위한 HTML 캔버스를 생성합니다.

마우스를 사용하여 표시된 사각형 안에 숫자를 그린 다음 **Classify**를 클릭합니다.

마우스로 사각형 안에 숫자를 적고 Classify 버튼을 눌러 분류 결과를 확인해 봅시다.

In [19]:
# Create an HTML canvas to evaluate the model
from IPython.display import HTML
import cv2
import numpy as np
import base64

def classify(img):
    img = base64.b64decode(img[len('data:image/png;base64,'):])
    img = cv2.imdecode(np.fromstring(img, np.uint8),-1)
    img = cv2.resize(img[:,:,3], (28,28))
    img = nd.array(img).as_in_context(ctx).reshape((-1, 784)).astype(np.float32)/255
    return int(nd.argmax(net(img), axis=1).asnumpy()[0])

HTML(filename = "mnist.html")